<a href="https://colab.research.google.com/github/jacquesescp/DSforAM_group10/blob/main/Draft_SharkRisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# app.py
import base64
import io
import traceback
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf

!pip install dash
!pip install dash-bootstrap-components

from dash import Dash, dcc, html, dash_table, Input, Output, State, callback_context

# =========================
# Black–Scholes Greek utils
# =========================
def _d1(S, K, T, r, sigma):
    if S <= 0 or K <= 0 or sigma <= 0 or T <= 0:
        return np.nan
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def _d2(d1, sigma, T):
    return d1 - sigma * np.sqrt(T)

def bs_greeks(S, K, T, r, sigma, opt_type):
    """
    Greeks per 1 option (not scaled by quantity/multiplier).
    Theta returned per day; Vega per 1 vol point (0.01).
    """
    if any(pd.isna(x) for x in [S, K, T, r, sigma]) or T <= 0 or sigma <= 0:
        return dict(delta=np.nan, gamma=np.nan, theta=np.nan, vega=np.nan, rho=np.nan)

    d1 = _d1(S, K, T, r, sigma)
    d2 = _d2(d1, sigma, T)
    if pd.isna(d1) or pd.isna(d2):
        return dict(delta=np.nan, gamma=np.nan, theta=np.nan, vega=np.nan, rho=np.nan)

    N, n = norm.cdf, norm.pdf
    if str(opt_type).lower() == "call":
        delta = N(d1)
        theta = (-(S * n(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * N(d2))
        rho   = K * T * np.exp(-r * T) * N(d2)
    else:
        delta = N(d1) - 1.0
        theta = (-(S * n(d1) * sigma) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * N(-d2))
        rho   = -K * T * np.exp(-r * T) * N(-d2)

    gamma = n(d1) / (S * sigma * np.sqrt(T))
    vega  = S * n(d1) * np.sqrt(T)

    return dict(
        delta=delta,
        gamma=gamma,
        theta=theta / 365.0,   # per day
        vega=vega / 100.0,     # per 1 vol point (0.01)
        rho=rho
    )

# =========================
# Data helpers
# =========================
def parse_portfolio(contents, filename):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    if filename.lower().endswith('.csv'):
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    else:
        raise ValueError("Please upload a CSV file.")
    return df

def ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ["symbol","asset_type","quantity","contract_multiplier","option_type","strike","expiry","implied_vol"]
    for c in cols:
        if c not in df.columns:
            df[c] = np.nan

    df["symbol"] = df["symbol"].astype(str).str.upper().str.strip()
    df["asset_type"] = df["asset_type"].astype(str).str.lower().str.strip()
    df["option_type"] = df["option_type"].astype(str).str.lower().str.strip()

    df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").fillna(0.0)
    df["contract_multiplier"] = pd.to_numeric(df["contract_multiplier"], errors="coerce").fillna(100.0)
    df["strike"] = pd.to_numeric(df["strike"], errors="coerce")
    df["implied_vol"] = pd.to_numeric(df["implied_vol"], errors="coerce")
    if "expiry" in df.columns:
        df["expiry"] = pd.to_datetime(df["expiry"], errors="coerce").dt.date
    return df

def fetch_spot_prices(symbols):
    prices = {}
    for sym in set(s for s in symbols if isinstance(s, str) and len(s) > 0):
        try:
            hist = yf.Ticker(sym).history(period="1d")
            prices[sym] = float(hist["Close"].iloc[-1]) if not hist.empty else np.nan
        except Exception:
            prices[sym] = np.nan
    return prices

def compute_portfolio(df: pd.DataFrame, r_annual: float):
    today = datetime.utcnow().date()
    syms = df["symbol"].dropna().unique().tolist()
    spot_map = fetch_spot_prices(syms)

    rows = []
    for _, row in df.iterrows():
        sym = row["symbol"]
        S = spot_map.get(sym, np.nan)
        qty = float(row["quantity"])
        mult = float(row["contract_multiplier"]) if not pd.isna(row["contract_multiplier"]) else 100.0
        asset_type = row["asset_type"]

        if asset_type == "stock":
            delta = 1.0; gamma = 0.0; theta = 0.0; vega = 0.0; rho = 0.0
        else:
            K = row["strike"]; iv = row["implied_vol"]; expiry = row["expiry"]
            if pd.isna(S) or pd.isna(K) or pd.isna(iv) or pd.isna(expiry):
                delta = gamma = theta = vega = rho = np.nan
            else:
                T = max((expiry - today).days, 0) / 365.0
                g = bs_greeks(S, K, T, r_annual, iv, row["option_type"])
                delta, gamma, theta, vega, rho = g["delta"], g["gamma"], g["theta"], g["vega"], g["rho"]

        scale = qty * mult
        rows.append({
            "symbol": sym, "asset_type": asset_type, "quantity": qty, "contract_multiplier": mult,
            "spot": S, "option_type": row.get("option_type",""), "strike": row.get("strike", np.nan),
            "expiry": row.get("expiry", np.nan), "implied_vol": row.get("implied_vol", np.nan),
            "delta": delta, "gamma": gamma, "theta": theta, "vega": vega, "rho": rho,
            "Delta (pos)": delta * scale,
            "Gamma (pos)": gamma * scale,
            "Theta/day (pos)": theta * scale,
            "Vega/pt (pos)": vega * scale,
            "Rho (pos)": rho * scale
        })

    out = pd.DataFrame(rows)
    totals = out[["Delta (pos)","Gamma (pos)","Theta/day (pos)","Vega/pt (pos)","Rho (pos)"]].sum(numeric_only=True)
    totals.name = "Portfolio totals"
    return out, totals

def shock_profile(df, r_annual, pct_moves=np.linspace(-0.1, 0.1, 21)):
    """Portfolio Delta across uniform spot shocks (IV held constant)."""
    today = datetime.utcnow().date()
    syms = df["symbol"].dropna().unique().tolist()
    base_spots = fetch_spot_prices(syms)

    opt_rows = df[df["asset_type"] == "option"].copy()
    stock_rows = df[df["asset_type"] == "stock"].copy()

    results = []
    for move in pct_moves:
        spot_map = {sym: (base_spots.get(sym, np.nan) * (1 + move) if not pd.isna(base_spots.get(sym, np.nan)) else np.nan)
                    for sym in set(syms)}
        total_delta = 0.0

        # Stocks
        for _, r in stock_rows.iterrows():
            mult = float(r.get("contract_multiplier", 1) or 1)
            total_delta += 1.0 * float(r["quantity"]) * mult

        # Options
        for _, r in opt_rows.iterrows():
            sym = r["symbol"]; S = spot_map.get(sym, np.nan)
            K = r["strike"]; iv = r["implied_vol"]; expiry = r["expiry"]
            if pd.isna(S) or pd.isna(K) or pd.isna(iv) or pd.isna(expiry):
                continue
            T = max((expiry - today).days, 0) / 365.0
            g = bs_greeks(S, K, T, r_annual, iv, r["option_type"])
            mult = float(r.get("contract_multiplier", 100) or 100)
            total_delta += g["delta"] * float(r["quantity"]) * mult

        results.append({"spot_move_pct": move * 100.0, "portfolio_delta": total_delta})

    return pd.DataFrame(results)

# ===========================================
# Live candidates (shares + option chains via yfinance)
# ===========================================
def live_option_candidates(symbol, r_annual, n_expiries=2, strikes_around_atm=1):
    """
    Build candidate hedges not necessarily in the current portfolio:
    - Shares (±1)
    - For the nearest expiries: ATM ± k strikes, both calls/puts (±1 contract)
    Uses yfinance IV if present, else fallback IV=0.25.
    """
    out = []

    # Spot
    try:
        hist = yf.Ticker(symbol).history(period="1d")
        if hist is None or hist.empty:
            return out
        S = float(hist["Close"].iloc[-1])
    except Exception:
        return out

    # Shares (unit = 1 share)
    out.append(dict(
        desc=f"{symbol} shares (±1)",
        symbol=symbol, type="stock", expiry=None, strike=None,
        unit_scale=1.0,
        greeks=np.array([1.0, 0.0, 0.0, 0.0, 0.0], dtype=float)
    ))

    # Options (unit = 1 contract, multiplier 100)
    try:
        tk = yf.Ticker(symbol)
        expiries = (tk.options or [])[:int(max(0, n_expiries))]
        for exp in expiries:
            chain = tk.option_chain(exp)
            for side, df_side in [("call", getattr(chain, "calls", None)),
                                  ("put", getattr(chain, "puts", None))]:
                if df_side is None or df_side.empty:
                    continue
                df_side = df_side.copy()
                df_side["dist"] = (df_side["strike"] - S).abs()
                df_side = df_side.sort_values("dist")

                k = int(max(0, strikes_around_atm))
                candidates = df_side.head(1 + 2*k)  # ATM ± k

                for _, opt in candidates.iterrows():
                    K = float(opt.get("strike", np.nan))
                    if not np.isfinite(K):
                        continue
                    iv = float(opt.get("impliedVolatility", np.nan))
                    if not (np.isfinite(iv) and iv > 0):
                        iv = 0.25
                    # time to expiry
                    try:
                        T = max((datetime.strptime(exp, "%Y-%m-%d").date() - datetime.utcnow().date()).days, 0) / 365.0
                    except Exception:
                        continue
                    g = bs_greeks(S, K, T, float(r_annual or 0.0), iv, side)
                    mult = 100.0
                    g_vec = np.array([g["delta"], g["gamma"], g["theta"], g["vega"], g["rho"]], dtype=float) * mult
                    g_vec = np.nan_to_num(g_vec, nan=0.0, posinf=0.0, neginf=0.0)
                    out.append(dict(
                        desc=f"{symbol} {side.upper()} {int(round(K))} {exp} (±1 contract)",
                        symbol=symbol, type=side, expiry=exp, strike=K,
                        unit_scale=1.0, greeks=g_vec
                    ))
    except Exception:
        pass

    return out

def build_live_candidates_for_portfolio(df, r_annual):
    """
    For all symbols present in the portfolio + hedge ETFs (SPY, QQQ, TLT),
    return a list of candidate trades with per-unit greek vectors.
    """
    syms = sorted(set(df["symbol"].dropna().tolist()))
    hedge_syms = ["SPY", "QQQ", "TLT"]  # Yahoo supports chains for these; avoid SPX/VIX
    actions = []
    for sym in syms + hedge_syms:
        actions.extend(live_option_candidates(sym, r_annual, n_expiries=5, strikes_around_atm=5))
    return actions

# =========================
# Solver (weighted least squares) + rounding
# =========================
def suggest_trades_to_neutralize(totals_vec, actions,
                                 max_abs_contracts=500,
                                 max_abs_shares=5000,
                                 share_round=10):
    if not actions:
        return []

    # Build A and b
    A = np.column_stack([a["greeks"] for a in actions]).astype(float)
    b = -np.asarray(totals_vec, dtype=float)

    # Sanitize NaN/Inf and remove all-zero columns
    A = np.nan_to_num(A, nan=0.0, posinf=0.0, neginf=0.0)
    b = np.nan_to_num(b, nan=0.0, posinf=0.0, neginf=0.0)
    keep = ~(np.all(A == 0.0, axis=0))
    if not np.any(keep):
        return []
    A = A[:, keep]
    kept_actions = [a for a, k in zip(actions, keep) if k]

    # Row weighting to equalize scales across Δ, Γ, Θ, Vega, ρ
    scales = []
    for i in range(A.shape[0]):
        row = np.abs(A[i, :])
        s = max(np.percentile(row[row > 0], 90) if np.any(row > 0) else 0.0,
                abs(b[i]),
                1e-9)
        scales.append(s)
    W = np.diag([1.0/s for s in scales])
    A_w = W @ A
    b_w = W @ b

    # Solve weighted LS
    try:
        x, *_ = np.linalg.lstsq(A_w, b_w, rcond=None)
    except Exception:
        return []

    # Practical rounding
    plan = []
    for xi, a in zip(x, kept_actions):
        if a["type"] == "stock":
            units = int(np.clip(np.round(xi / share_round) * share_round,
                                -max_abs_shares, max_abs_shares))
        else:
            units = int(np.clip(np.round(xi), -max_abs_contracts, max_abs_contracts))
        if units != 0:
            plan.append((a, units))

    # Fallback if everything rounded to zero
    if not plan:
        target_idx = int(np.argmax(np.abs(b)))  # 0=Δ,1=Γ,2=Θ,3=Vega,4=ρ
        col = int(np.argmax(np.abs(A[target_idx, :])))
        a = kept_actions[col]
        units = 100 if a["type"] == "stock" else 5
        sign = 1 if np.sign(A[target_idx, col]) == np.sign(b[target_idx]) else -1
        plan = [(a, sign * units)]

    return plan

# ---------- helper: compute plan effect on portfolio greeks ----------
GREEK_LABELS = ["Δ","Γ","Θ/day","Vega/pt","ρ"]

def plan_effect_vector(plan):
    """Sum of (units * greeks) over actions."""
    if not plan:
        return np.zeros(5, dtype=float)
    vecs = [a["greeks"] * units for (a, units) in plan]
    return np.sum(vecs, axis=0)

def impacted_greeks(before, effect, after, top_k=3):
    """Return list of greek names most reduced in absolute value by the plan."""
    reduction = np.abs(before) - np.abs(after)
    idx = np.argsort(-np.abs(reduction))[:top_k]
    return [GREEK_LABELS[i] for i in idx if np.isfinite(reduction[i])]

# =========================
# Dash UI
# =========================
app = Dash(__name__)
app.title = "Portfolio Greeks Dashboard"

app.layout = html.Div([
    html.H2("Portfolio Greeks Dashboard"),
    html.P("Upload a CSV of positions (stocks + options)."),

    dcc.Upload(
        id='upload-data',
        children=html.Div(['Drag and drop or ', html.A('Select Portfolio CSV')]),
        style={'width': '100%', 'height': '60px', 'lineHeight': '60px',
               'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '6px',
               'textAlign': 'center', 'margin': '10px 0'}
    ),

    html.Div([
        html.Label("Risk-free rate (annual, e.g., 0.045 for 4.5%)"),
        dcc.Input(id='riskfree', type='number', value=0.04, step=0.001, style={'width':'220px', 'marginRight':'12px'}),
        html.Button("Recalculate", id='recalc', n_clicks=0)
    ], style={'marginBottom': '10px'}),

    html.Div(id='file-info', style={'marginBottom': '10px', 'fontStyle': 'italic'}),

    dash_table.DataTable(
        id='positions-table',
        columns=[{"name": c, "id": c} for c in
                 ["symbol","asset_type","quantity","contract_multiplier","spot",
                  "option_type","strike","expiry","implied_vol",
                  "Delta (pos)","Gamma (pos)","Theta/day (pos)","Vega/pt (pos)","Rho (pos)"]],
        data=[],
        filter_action='native',
        sort_action='native',
        page_size=12,
        style_table={'overflowX': 'auto'},
        style_cell={'fontFamily': 'monospace', 'fontSize': 13, 'padding': '6px'}
    ),

    html.H4("Portfolio totals"),
    html.Div(id='totals', style={'marginBottom':'14px'}),

    html.H4("Risk Mitigation (Greeks Neutralizer)"),
    html.Div([
        html.Button("Suggest neutralizing trades", id="suggest-btn", n_clicks=0),
        html.Span("  (uses live option chains + weighted least squares; no paid API)", style={"marginLeft":"8px", "fontStyle":"italic"})
    ], style={"marginBottom":"8px"}),
    html.Div(id="suggestions", style={"whiteSpace":"pre-wrap", "fontFamily":"monospace"}),

    html.H4("Delta vs Spot Shock"),
    dcc.Graph(id='shock-graph')
])

# =========================
# Callbacks
# =========================
@app.callback(
    [Output('positions-table', 'data'),
     Output('totals', 'children'),
     Output('file-info', 'children'),
     Output('shock-graph', 'figure')],
    [Input('upload-data', 'contents'),
     Input('recalc', 'n_clicks')],
    [State('upload-data', 'filename'),
     State('riskfree', 'value'),
     State('positions-table', 'data')]
)
def update_output(contents, n_clicks, filename, r_annual, existing_data):
    triggered = [t['prop_id'] for t in (callback_context.triggered or [])]
    if contents is None and not existing_data:
        return [], "", "No file uploaded.", dict(data=[], layout={})

    if contents is not None and ("upload-data.contents" in "".join(triggered)):
        try:
            df = parse_portfolio(contents, filename)
            df = ensure_columns(df)
        except Exception as e:
            return [], "", f"Upload error: {e}", dict(data=[], layout={})
    else:
        df = pd.DataFrame(existing_data)
        df = ensure_columns(df)

    # Compute Greeks
    r = float(r_annual or 0.0)
    positions, totals = compute_portfolio(df.copy(), r)

    # Totals as bullet list
    totals_html = html.Ul([
        html.Li(f"Delta (pos): {totals.get('Delta (pos)',0):,.2f}"),
        html.Li(f"Gamma (pos): {totals.get('Gamma (pos)',0):,.6f}"),
        html.Li(f"Theta/day (pos): {totals.get('Theta/day (pos)',0):,.2f}"),
        html.Li(f"Vega/pt (pos): {totals.get('Vega/pt (pos)',0):,.2f}"),
        html.Li(f"Rho (pos): {totals.get('Rho (pos)',0):,.2f}")
    ])

    # Shock profile chart
    prof = shock_profile(df.copy(), r)
    fig = {
        "data": [{
            "x": prof["spot_move_pct"],
            "y": prof["portfolio_delta"],
            "mode": "lines+markers",
            "name": "Portfolio Δ"
        }],
        "layout": {
            "xaxis": {"title": "Spot move (%)"},
            "yaxis": {"title": "Portfolio Delta (scaled)"},
            "margin": {"l": 60, "r": 20, "t": 10, "b": 50},
            "hovermode": "x unified"
        }
    }

    file_msg = f"Loaded: {filename} — {len(positions)} rows."
    return positions.to_dict("records"), totals_html, file_msg, fig

@app.callback(
    Output("suggestions", "children"),
    Input("suggest-btn", "n_clicks"),
    State("positions-table", "data"),
    State("riskfree", "value")
)
def make_suggestions(n_clicks, table_data, r_annual):
    if not n_clicks or not table_data:
        return ""

    try:
        df = pd.DataFrame(table_data or [])
        df = ensure_columns(df)

        r = float(r_annual or 0.0)
        positions, totals = compute_portfolio(df.copy(), r)
        before_vec = np.array([
            float(totals.get("Delta (pos)", 0.0)),
            float(totals.get("Gamma (pos)", 0.0)),
            float(totals.get("Theta/day (pos)", 0.0)),
            float(totals.get("Vega/pt (pos)", 0.0)),
            float(totals.get("Rho (pos)", 0.0))
        ], dtype=float)

        actions = build_live_candidates_for_portfolio(df, r)

        lines = []
        lines.append(f"Candidates available: {len(actions)}")

        if not actions:
            lines.append("No candidate trades could be formed (live option chains unavailable).")
            return "\n".join(lines)

        plan = suggest_trades_to_neutralize(before_vec, actions)

        # ---- Report current totals
        lines.append("Current totals:")
        lines.append(
            f"  Δ={before_vec[0]:,.2f}  Γ={before_vec[1]:,.6f}  Θ/day={before_vec[2]:,.2f}  "
            f"Vega/pt={before_vec[3]:,.2f}  ρ={before_vec[4]:,.2f}"
        )

        if not plan:
            lines.append("\nGreeks already close to zero under the current candidate set.")
            return "\n".join(lines)

        # ---- Show plan
        lines.append("\nSuggested trades (integer units; + = buy, − = sell):")
        for a, u in plan:
            sign = "+" if u > 0 else ""
            lines.append(f"  {sign}{int(u)} × {str(a.get('desc',''))}")

        # ---- Compute plan effect and after totals
        effect_vec = plan_effect_vector(plan)  # same order as GREEK_LABELS
        after_vec = before_vec + effect_vec

        def fmtrow(tag, vec):
            return (f"{tag}: "
                    f"Δ={vec[0]:,.2f}  Γ={vec[1]:,.6f}  Θ/day={vec[2]:,.2f}  "
                    f"Vega/pt={vec[3]:,.2f}  ρ={vec[4]:,.2f}")

        lines.append("\n--- Impact of implementation ---")
        lines.append(fmtrow("Adjustment (from suggested trades)", effect_vec))
        lines.append(fmtrow("After implementation (simulated)", after_vec))

        # ---- Which greeks are most impacted?
        impacted = impacted_greeks(before_vec, effect_vec, after_vec, top_k=3)
        if impacted:
            lines.append(f"Greeks most impacted: {', '.join(impacted)}")

        lines.append("\nNotes:")
        lines.append("  • Uses yfinance option chains + weighted least squares; no paid API.")
        lines.append("  • 'After implementation' assumes current spot/IV; live markets will differ.")
        lines.append("  • Rounding to whole shares/contracts can leave small residual Greeks.")
        return "\n".join(lines)

    except Exception:
        return "Error in suggestion engine:\n" + traceback.format_exc()

# =========================
# Run locally (safe)
# =========================
if __name__ == "__main__":
    app.run(debug=True, host="127.0.0.1", port=8050)